# MNIST data classification: Premade estimator and input pipeline

This notebook shows how to use input pipelines with a premade estimator. You can monitor the training live with tensorboard!
```
tensorboard --logdir=/path/to/summary
```

In [1]:
import numpy as np
import matplotlib
import matplotlib.pyplot as plt
import tensorflow as tf

In [2]:
tf.VERSION

'1.12.0'

In [3]:
def decode(serialized_example):
    """Parses an image and label from the given `serialized_example`."""
    features = tf.parse_single_example(
        serialized_example,
        features={
            'image_raw': tf.FixedLenFeature([], tf.string),
            'label': tf.FixedLenFeature([], tf.int64),
        })
    label = tf.cast(features['label'], tf.int32)
    image = tf.decode_raw(features['image_raw'], tf.uint8)
    return image, label


def normalize(image, label):
    """Convert `image` from [0, 255] -> [-0.5, 0.5] floats."""
    image = tf.cast(image, tf.float32) * (1. / 255) - 0.5
    return {'img': image}, label


def train_input_fn():
    """Reads input data num_epochs times."""
    dataset = tf.data.TFRecordDataset('train.tfrecords')
    dataset = dataset.map(decode)
    dataset = dataset.map(normalize)
    dataset = dataset.shuffle(6000)
    dataset = dataset.batch(256)
    dataset = dataset.repeat(5)
    
    iterator = dataset.make_one_shot_iterator()
    image, label = iterator.get_next()
    return image, label

In [4]:
img = tf.feature_column.numeric_column('img', dtype=tf.float64, shape=(784,))

In [5]:
estimator = tf.estimator.DNNClassifier(feature_columns=[img],
                                       hidden_units=[256, 32],
                                       optimizer=tf.train.AdamOptimizer(1e-4),
                                       n_classes=10,
                                       model_dir='./summary')

INFO:tensorflow:Using default config.
INFO:tensorflow:Using config: {'_model_dir': './summary', '_tf_random_seed': None, '_save_summary_steps': 100, '_save_checkpoints_steps': None, '_save_checkpoints_secs': 600, '_session_config': allow_soft_placement: true
graph_options {
  rewrite_options {
    meta_optimizer_iterations: ONE
  }
}
, '_keep_checkpoint_max': 5, '_keep_checkpoint_every_n_hours': 10000, '_log_step_count_steps': 100, '_train_distribute': None, '_device_fn': None, '_protocol': None, '_eval_distribute': None, '_experimental_distribute': None, '_service': None, '_cluster_spec': <tensorflow.python.training.server_lib.ClusterSpec object at 0x7f61184c3fd0>, '_task_type': 'worker', '_task_id': 0, '_global_id_in_cluster': 0, '_master': '', '_evaluation_master': '', '_is_chief': True, '_num_ps_replicas': 0, '_num_worker_replicas': 1}


In [6]:
! rm -r summary/

In [7]:
train = estimator.train(input_fn=train_input_fn)

INFO:tensorflow:Calling model_fn.
INFO:tensorflow:Done calling model_fn.
INFO:tensorflow:Create CheckpointSaverHook.
INFO:tensorflow:Graph was finalized.
INFO:tensorflow:Running local_init_op.
INFO:tensorflow:Done running local_init_op.
INFO:tensorflow:Saving checkpoints for 0 into ./summary/model.ckpt.
INFO:tensorflow:loss = 609.24274, step = 1
INFO:tensorflow:global_step/sec: 53.1232
INFO:tensorflow:loss = 283.1012, step = 101 (1.883 sec)
INFO:tensorflow:global_step/sec: 56.9028
INFO:tensorflow:loss = 168.08698, step = 201 (1.758 sec)
INFO:tensorflow:global_step/sec: 56.4218
INFO:tensorflow:loss = 163.23788, step = 301 (1.772 sec)
INFO:tensorflow:global_step/sec: 56.5201
INFO:tensorflow:loss = 114.52593, step = 401 (1.769 sec)
INFO:tensorflow:global_step/sec: 57.1656
INFO:tensorflow:loss = 81.08363, step = 501 (1.749 sec)
INFO:tensorflow:global_step/sec: 54.1802
INFO:tensorflow:loss = 105.92674, step = 601 (1.846 sec)
INFO:tensorflow:global_step/sec: 63.6748
INFO:tensorflow:loss = 63

In [8]:
estimator.get_variable_names()

['dnn/head/beta1_power',
 'dnn/head/beta2_power',
 'dnn/hiddenlayer_0/bias',
 'dnn/hiddenlayer_0/bias/t_0/Adam',
 'dnn/hiddenlayer_0/bias/t_0/Adam_1',
 'dnn/hiddenlayer_0/kernel',
 'dnn/hiddenlayer_0/kernel/t_0/Adam',
 'dnn/hiddenlayer_0/kernel/t_0/Adam_1',
 'dnn/hiddenlayer_1/bias',
 'dnn/hiddenlayer_1/bias/t_0/Adam',
 'dnn/hiddenlayer_1/bias/t_0/Adam_1',
 'dnn/hiddenlayer_1/kernel',
 'dnn/hiddenlayer_1/kernel/t_0/Adam',
 'dnn/hiddenlayer_1/kernel/t_0/Adam_1',
 'dnn/logits/bias',
 'dnn/logits/bias/t_0/Adam',
 'dnn/logits/bias/t_0/Adam_1',
 'dnn/logits/kernel',
 'dnn/logits/kernel/t_0/Adam',
 'dnn/logits/kernel/t_0/Adam_1',
 'global_step']